In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/trash/detectors/detectors_r50_1x_trash.py')

PREFIX = '../../input/data/'



In [3]:
# cfg.log_config.hooks[1].init_kwargs.project = project_name
# cfg.log_config.hooks[1].init_kwargs.name = wandb.run.name
cfg.optimizer = dict(type = 'Adam', lr = 0.0001)
cfg.lr_config.step = [13, 17]
cfg.runner.max_epochs = 20
# cfg.runner.max_epochs = 4

In [4]:
##### albu_augmentation
albu_train_transforms = [
    dict(
        type='Resize',
        width=512,
        height=512,
        p=1.0),
    dict(
        type='Rotate',
        limit=10,
        p=0.5), 
    dict(
        type='RandomBrightnessContrast',
        p=1.0),
]
#####

In [5]:
# img_norm_cfg
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Pad', size_divisor=32),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels'],
        meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
                   'pad_shape', 'scale_factor')
    )
]

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)


cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
# 새롭게 만든 모델을 저장할 장소
cfg.work_dir = './work_dirs/detectors_r50_1x_trash'

# class 개수 11개로 수정
cfg.model.roi_head.bbox_head[0].num_classes = 11
cfg.model.roi_head.bbox_head[1].num_classes = 11
cfg.model.roi_head.bbox_head[2].num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
# checkpoint_path = os.path.join(cfg.work_dir, 'best_bbox_mAP_50.pth')
checkpoint_path = os.path.join(cfg.work_dir, 'latest.pth')

In [6]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-19 03:42:41,326 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-19 03:42:41,327 - mmdet - INFO - Use load_from_torchvision loader
2021-05-19 03:42:41,613 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

In [8]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 7.3 task/s, elapsed: 115s, ETA:     0s

In [9]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_35.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,8 0.9999918 178.61235 237.34521 459.38416 469....,batch_01_vt/0021.jpg
1,8 0.99994314 4.08609 245.13318 293.04123 455.6...,batch_01_vt/0028.jpg
2,1 0.99338984 72.28044 169.7759 374.0547 379.90...,batch_01_vt/0031.jpg
3,8 0.99182254 43.98828 127.27782 388.68237 373....,batch_01_vt/0032.jpg
4,0 0.07617544 37.997246 457.81512 71.59295 473....,batch_01_vt/0070.jpg
